# 🐈 Cat Discriminator

### 📝 Imports

In [1]:
import torch
import torch.optim as optim
from torch.utils.data import DataLoader

import albumentations as A

import torchvision.transforms as transforms

import os

from src.cat_discriminator_neural_net import CatDiscriminatorNeuralNet

from src.augmentation.data_augmenter import DataAugmenter
from src.cats_dataset import CatsDataset

c:\Users\Dan\AppData\Local\Programs\Python\Python312\Lib\site-packages\albumentations\__init__.py:24: UserWarning: A new version of Albumentations is available: 2.0.2 (you have 1.4.23). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


### 🔧 Config

In [2]:
image_size = 512

learning_rate = 0.01
batches_between_saves = 5
number_of_batches = 30

batch_size = 32

num_cores = 2

saved_model_path = "trained_networks/cat_discriminator.pth"

training_data_path = 'data/training/'

### 🌐 Create Transforms

In [3]:
transforms = transforms.Compose([
    DataAugmenter(image_size, augment_images=True),
    transforms.ToTensor(),
])

### 🚦 Load Training Data

In [4]:
dataset = CatsDataset(
    root_dir=training_data_path,
    transform=transforms
)

dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, num_workers=num_cores)

### 🥾 Initialize the Neural Net

In [5]:
net = CatDiscriminatorNeuralNet(learning_rate, saved_model_path)
net.cuda();

### 🏃‍♂️‍➡️ Train

In [6]:
batch_number = 0

while batch_number < number_of_batches:
    print('Training batch: ', batch_number)
    net.run_training_batch(learning_rate, dataloader)

    if batch_number % batches_between_saves == 0:
        print('Saving model...')
        torch.save(net.state_dict(), saved_model_path)

    batch_number += 1


Training batch:  0
Loss: 0.1286
Saving model...
Training batch:  1
Loss: 0.1318
Training batch:  2


KeyboardInterrupt: 

### 💾 Save Progress

In [34]:
torch.save(net.state_dict(), saved_model_path)